In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# tag: remove-cell applied

# Dichotomous Data with Bayesian Model Averaging

Bayesian model averaging is currently available for dichotomous datasets in `pybmds`. Here, we describe how to run a dichotomous analysis with Bayesian model averaging and how to plot your results. Also, we will demonstrate how you can override the default priors for parameter estimation.

## Single model fit

You can run one model with the default prior distributions for the parameters, rather than running all of the models and finding the model average. 

To run the Logistic model with the default prior distributions, see below. Note the model outputs show the prior distributions for `a ~ Normal(0, 2)`, `b ~ Lognormal(0, 2)`, and their constraints:

In [ ]:
import pybmds
from pybmds.models import dichotomous

# create a dichotomous dataset
dataset = pybmds.DichotomousDataset(
    doses=[0, 25, 75, 125, 200],
    ns=[20, 20, 20, 20, 20],
    incidences=[0, 1, 7, 15, 19],
)

model = dichotomous.Logistic(
    dataset=dataset, 
    settings={"priors": pybmds.PriorClass.bayesian}
)
result = model.execute()
print(model.text())

### Change parameter settings

You can change the input parameter settings for an analysis.  

Running a single Bayesian model with base configuration as an example:

In [ ]:
model = dichotomous.Logistic(
    dataset=dataset, settings={"priors": pybmds.PriorClass.bayesian}
)

print(model.settings.tbl())
print(model.priors_tbl())

Configuring the model settings or priors:

In [ ]:
model = dichotomous.Logistic(
    dataset=dataset,
    settings={
        "priors": pybmds.PriorClass.bayesian,
        "bmr": 0.05
    }
)
model.settings.priors.update('a', stdev=1, min_value=-15, max_value=15)
model.settings.priors.update('b', stdev=3)

print(model.settings.tbl())
print(model.priors_tbl())

You can also change the parameter prior types from the default listed in the BMDS User Guide. Parameters can be given a Normal, Log-Normal, or Uniform distribution. These can be changed by:

In [ ]:
model = dichotomous.Weibull(
    dataset=dataset,
    settings={
        "priors": pybmds.PriorClass.bayesian,
        "bmr": 0.05,
    }
)
model.settings.priors.update(
    'g', type=pybmds.PriorDistribution.Uniform, min_value=0, max_value=1
)
print(model.priors_tbl())

Or:

In [ ]:
model.settings.priors.update(
    'g', 
    type=pybmds.PriorDistribution.Lognormal, 
    min_value=0, 
    max_value=1, 
    initial_value=0, 
    stdev=1.5,
)
print(model.priors_tbl())

## Multiple model fit (sessions)

To run Bayesian model averaging:

In [ ]:
import pybmds
from pybmds.models import dichotomous

# create a dichotomous dataset
dataset = pybmds.DichotomousDataset(
    doses=[0, 25, 75, 125, 200],
    ns=[20, 20, 20, 20, 20],
    incidences=[0, 1, 7, 15, 19],
)

session = pybmds.Session(dataset=dataset)
session.add_default_bayesian_models()
session.execute()

res = session.model_average.results
print(f"BMD = {res.bmd:.2f} [{res.bmdl:.2f}, {res.bmdu:.2f}]")

session.plot(colorize=False)

### Change session settings

For a dichotomous Bayesian model average session, the default settings use a BMR of 10% Extra Risk and a 95% confidence interval. To change these settings:

In [ ]:
session = pybmds.Session(dataset=dataset)
session.add_default_bayesian_models(
    settings = {
        "bmr": 0.05,
        "bmr_type": pybmds.DichotomousRiskType.AddedRisk,
        "alpha": 0.1
    }
)

This would run the dichotomous models for a BMR of 5% Added Risk at a 90% confidence interval.

### Run subset of models

Instead of running all available dichotomous models, you can select a subset of models. 

For example, to model average on the Logistic, Probit, and Weibull models:

In [ ]:
session = pybmds.Session(dataset=dataset)
session.add_model(pybmds.Models.Weibull, {"priors": pybmds.PriorClass.bayesian})
session.add_model(pybmds.Models.Logistic, {"priors": pybmds.PriorClass.bayesian})
session.add_model(pybmds.Models.Probit, {"priors": pybmds.PriorClass.bayesian})
session.add_model_averaging()

session.execute()

res = session.model_average.results
print(f"BMD = {res.bmd:.2f} [{res.bmdl:.2f}, {res.bmdu:.2f}]")

session.plot(colorize=False)